In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

__Домашняя работа 3.__

In [2]:
df = pd.read_csv('train_case2.csv', ';')
df.head(7)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
5,8,21914,1,151,67.0,120,80,2,2,0,0,0,0
6,9,22113,1,157,93.0,130,80,3,1,0,0,1,0


In [3]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

In [4]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [5]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

__Задание 1.__ обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
при обучении моделей обязательно использовать кроссвалидацию

In [6]:
overall_mark = pd.DataFrame(columns=['type', 'precision', 'recall', 'f_score', 'roc auc'])

In [7]:
def get_scores(y_test, preds, mtype):
    e = 1e-8 # Возможно деление на ноль
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall + e)
    ix = np.argmax(fscore)
    mark = pd.DataFrame([[mtype, precision[ix], recall[ix], fscore[ix], roc_auc_score(y_test, preds)]], columns=['type', 'precision', 'recall', 'f_score', 'roc auc'])
    return mark
    #overall_mark.append(mark)

In [8]:
Logreg = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])

In [9]:
Boosting = Pipeline([
    ('features',feats),
    ('classifier', AdaBoostClassifier(random_state = 42)),
])

In [10]:
RandForest = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

In [11]:
%%time
cv_scores_logreg = cross_val_score(Logreg, X_train, y_train, cv=16, scoring='roc_auc')
cv_score_logreg = np.mean(cv_scores_logreg)
std_logreg = np.std(cv_scores_logreg)
print('CV score is {}+-{}'.format(cv_score_logreg, std_logreg))

CV score is 0.7867401104915408+-0.00852135511666111
Wall time: 8.52 s


In [12]:
%%time
cv_scores_boosting = cross_val_score(Boosting, X_train, y_train, cv=16, scoring='roc_auc')
cv_score_boosting = np.mean(cv_scores_boosting)
std_boosting = np.std(cv_scores_boosting)
print('CV score is {}+-{}'.format(cv_score_boosting, std_boosting))

CV score is 0.7952419477681905+-0.0071441451920620375
Wall time: 50.7 s


In [13]:
%%time
cv_scores_forest = cross_val_score(RandForest, X_train, y_train, cv=16, scoring='roc_auc')
cv_score_forest = np.mean(cv_scores_forest)
std_forest = np.std(cv_scores_forest)
print('CV score is {}+-{}'.format(cv_score_forest, std_forest))

CV score is 0.7734501681056019+-0.007171140345435727
Wall time: 2min 7s


In [14]:
%%time
#обучим пайплайн на всем тренировочном датасете
Logreg.fit(X_train, y_train)
Boosting.fit(X_train, y_train)
RandForest.fit(X_train, y_train)

Wall time: 8.08 s


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='age')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('height',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='height')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('weight',
                                                 Pipeline(steps=[('selector',
                                        

In [15]:
preds = Logreg.predict_proba(X_test)[:, 1]
overall_mark = overall_mark.append(get_scores(y_test, preds, 'Logreg'))

In [16]:
preds = Boosting.predict_proba(X_test)[:, 1]
overall_mark = overall_mark.append(get_scores(y_test, preds, 'Boosting'))

In [17]:
preds = RandForest.predict_proba(X_test)[:, 1]
overall_mark = overall_mark.append(get_scores(y_test, preds, 'RandForest'))

__Задание 2.__ вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)

In [18]:
overall_mark

,type,precision,recall,f_score,roc auc
0,Logreg,0.647431,0.837558,0.730323,0.784035
0,Boosting,0.692471,0.789401,0.737766,0.794572
0,RandForest,0.642669,0.815553,0.718863,0.771037


__Задание 3.__ сделать выводы о том, какая модель справилась с задачей лучше других

На основе полученных метрик можно сделать вывод, что лучше всего себя показал AdaBoosting.